## Get Firmwares

Notebook to download pre-built firmwares published on the micropython download pages


In [1]:
# Below are the requirements for the notebook to run.
# %pip install BeautifulSoup4 requests

In [2]:
import requests
import re
import functools
from urllib.parse import urljoin

from typing import List, Dict
from bs4 import BeautifulSoup

In [3]:
# use functools.lru_cache to avoid needing to download pages multiple times
@functools.lru_cache(maxsize=500)
def get_page(page_url) -> str:
    """Get the HTML of a page and return it as a string."""
    response = requests.get(page_url)
    downloads_html = response.content.decode()
    return downloads_html


@functools.lru_cache(maxsize=500)
def get_board_urls(page_url) -> List[Dict[str, str]]:
    """Get the urls to all the board pages listed on this page."""
    downloads_html = get_page(page_url)
    soup = BeautifulSoup(downloads_html, "html.parser")
    tags = soup.findAll("a", recursive=True, attrs={"class": "board-card"})
    # assumes that all links are relative to the page url
    boards = [tag.get("href") for tag in tags]
    if "?" in page_url:
        page_url = page_url.split("?")[0]
    return [{"board": board, "url": page_url + board} for board in boards]

In [4]:
def firmware_list(board_url: str, base_url: str, ext: str) -> List[str]:
    """Get the urls to all the firmware files for a board."""
    html = get_page(board_url)
    soup = BeautifulSoup(html, "html.parser")
    # get all the a tags
    #   that have a url that starts with `/resources/firmware/`
    #   and ends with a matching extension
    tags = soup.findAll(
        "a",
        recursive=True,
        attrs={"href": re.compile(r"^/resources/firmware/.*\." + ext.lstrip(".") + "$")},
    )
    if "?" in base_url:
        base_url = base_url.split("?")[0]
    links = [urljoin(base_url, tag.get("href")) for tag in tags]
    return links

In [11]:
MICROPYTHON_ORG_URL = "https://micropython.org/"
# ports and firmware extensions we are interested in
PORT_FWTYPES = {
    "stm32": ".hex",
    "esp32": ".bin",
    "rp2": ".uf2",
    "samd": ".uf2",
}
# boards we are interested in ( this avoids getting a lot of boards we don't care about)
RELEVANT_BOARDS = [
    "PYBV11",
    "ESP32_GENERIC",
    "ESP32_GENERIC_S3",
    "RPI_PICO",
    "RPI_PICO_W",
    "ARDUINO_NANO_RP2040_CONNECT",
    "PIMORONI_PICOLIPO_16MB",
    "SEEED_WIO_TERMINAL",
]


# The first run takes ~60 seconds to run for 4 ports , all boards
# so it makes sense to cache the results and skip boards as soon as possible
def get_boards(fw_types, RELEVANT_BOARDS):
    board_urls = []
    for port in fw_types.keys():
        download_page_url = f"{MICROPYTHON_ORG_URL}download/?port={port}"
        _urls = get_board_urls(download_page_url)
        # filter out boards we don't care about
        _urls = [board for board in _urls if board["board"] in RELEVANT_BOARDS]
        # add the port to the board urls
        for board in _urls:
            board["port"] = port

        for board in _urls:
            # add a board to the list for each firmware found
            firmwares = firmware_list(board["url"], MICROPYTHON_ORG_URL, fw_types[port])
            for _url in firmwares:
                board["firmware"] = _url
                board["preview"] = "preview" in _url  # type: ignore
                # get version number and optional preview from url
                # 'PYBD_SF2-20231009-v1.22.0-preview.3.ga06f4c8df.hex' -> v1.22.0-preview.3
                # 'PYBD_SF2-20230426-v1.20.0.hex' -> v1.20.0
                ver_match = re.search(r"(\d+\.\d+\.\d+(-\w+.\d+)?)", _url)
                if ver_match:
                    board["version"] = ver_match.group(1)
                else:
                    board["version"] = ""

                board_urls.append(board.copy())
    return board_urls


board_urls = get_boards(PORT_FWTYPES, RELEVANT_BOARDS)

print(f"Found {len(board_urls)} firmwares")

Found 169 firmwares


In [12]:
relevant = [
    board
    for board in board_urls
    if board["board"] in RELEVANT_BOARDS and (board["version"] in ["1.22.0"] or board["preview"])
    # and b["port"] in ["esp32", "rp2"]
]
# relevant
print(f"Found {len(relevant)} relevant firmwares")

Found 85 relevant firmwares


In [17]:
# download the relevant files to the firmware folder
import requests
from pathlib import Path

firmware_folder = Path("firmware")
firmware_folder.mkdir(exist_ok=True)

for board in relevant:
    date_re = r"(-\d{8}-)"
    hash_re = r"(.g[0-9a-f]+\.)"
    # remove date from firmware name
    fname = re.sub(date_re, "-", Path(board["firmware"]).name)
    # remove hash from firmware name
    fname = re.sub(hash_re, ".", fname)
    filename = firmware_folder / board["port"] / fname
    filename.parent.mkdir(exist_ok=True)
    if filename.exists():
        print(f" {filename} already exists, skip download")
        continue
    print(f"Downloading {board['firmware']} to {filename}")
    r = requests.get(board["firmware"], allow_redirects=True)
    with open(filename, "wb") as f:
        f.write(r.content)
    board["filename"] = filename

 firmware\stm32\PYBV11-v1.22.0.hex already exists, skip download
 firmware\stm32\PYBV11-v1.22.0-preview.289.hex already exists, skip download
 firmware\stm32\PYBV11-v1.22.0-preview.283.hex already exists, skip download
 firmware\stm32\PYBV11-v1.22.0-preview.281.hex already exists, skip download
 firmware\stm32\PYBV11-v1.22.0-preview.278.hex already exists, skip download
 firmware\stm32\PYBV11-DP-v1.22.0.hex already exists, skip download
 firmware\stm32\PYBV11-DP-v1.22.0-preview.289.hex already exists, skip download
 firmware\stm32\PYBV11-DP-v1.22.0-preview.283.hex already exists, skip download
 firmware\stm32\PYBV11-DP-v1.22.0-preview.281.hex already exists, skip download
 firmware\stm32\PYBV11-DP-v1.22.0-preview.278.hex already exists, skip download
 firmware\stm32\PYBV11-NETWORK-v1.22.0.hex already exists, skip download
 firmware\stm32\PYBV11-NETWORK-v1.22.0-preview.289.hex already exists, skip download
 firmware\stm32\PYBV11-NETWORK-v1.22.0-preview.283.hex already exists, skip downl

In [18]:
!mpremote devs

COM4 017154B9 10c4:ea60 Silicon Labs None


In [1]:
serialport = "COM6"
# !esptool --chip esp32 --port {serialport} erase_flash
# !esptool --chip esp32 --port {serialport} write_flash -z 0x1000 "firmware\esp32\ESP32_GENERIC-SPIRAM-v1.22.0.bin"
!esptool --chip esp32 --port {serialport} write_flash -z 0x1000 "firmware\esp32\ESP32_GENERIC-SPIRAM-v1.22.0-preview.289.bin"

esptool.py v4.6.2
Serial port COM6
Connecting......
Chip is ESP32-D0WD (revision v1.0)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: b4:e6:2d:df:40:8d
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Flash will be erased from 0x00001000 to 0x0018bfff...
Compressed 1614624 bytes to 1055696...
Writing at 0x00001000... (1 %)
Writing at 0x00010a5e... (3 %)
Writing at 0x00018790... (4 %)
Writing at 0x000205a9... (6 %)
Writing at 0x0002684e... (7 %)
Writing at 0x0002fbb1... (9 %)
Writing at 0x00039936... (10 %)
Writing at 0x0003f7ce... (12 %)
Writing at 0x0004bc0d... (13 %)
Writing at 0x000515d9... (15 %)
Writing at 0x00056c57... (16 %)
Writing at 0x0005bf86... (18 %)
Writing at 0x00061230... (20 %)
Writing at 0x00066429... (21 %)
Writing at 0x0006b602... (23 %)
Writing at 0x00070ab8... (24 %)
Writing at 0x00075967... (26 %)
Writing at 0x0007aa31... (27 %)
Writing at 0x00081627... (29 %)
Writing at 0x0008